In [91]:
achievements = [
    {"title": "wandering shroom", "stam_cost": 15, "wing_cost": 2, "remaining": 20},
    {"title": "catoblepas anemos", "stam_cost": 20, "wing_cost": 2, "remaining": 20},
    {"title": "eolian phantom", "stam_cost": 25, "wing_cost": 2, "remaining": 18},
    {"title": "proud manticore", "stam_cost": 25, "wing_cost": 2, "remaining": 20},
    {"title": "ebon chimera", "stam_cost": 30, "wing_cost": 3, "remaining": 8}
]

In [92]:
from datetime import datetime

time_remaining = datetime(2020, 2, 27) - datetime.now()
hours_remaining, remainder = divmod(time_remaining.seconds, 3600)
hours_remaining += time_remaining.days * 24
minutes, seconds = divmod(remainder, 60)
print(hours_remaining, minutes, seconds)

150 47 41


In [99]:
current_stam = 26
current_wings = 4
max_stam = 119

# Real regen: 10/h; Honey Tea: 2/h
total_remaining_stam = current_stam + 12 * hours_remaining + minutes // 6 + minutes // 30
total_remaining_stam

1834

In [94]:
cost_to_finish = sum([i['remaining'] * i['stam_cost'] for i in achievements])
cost_to_finish

1890

In [95]:
total_difference = cost_to_finish - total_remaining_stam
total_difference

56

In [96]:
[i["title"] for i in achievements]

['wandering shroom',
 'catoblepas anemos',
 'eolian phantom',
 'proud manticore',
 'ebon chimera']

In [101]:
f = [i["title"] for i in achievements]
double_drops = [
    [0, 1, 2, 3, 4],
    [0, ],
    [1,3,],
    [0,],
    [4],
    [1,3],
    [2,],
    [0,1,2,3,4]
]

double_drops = [[f[j] for j in i] for i in double_drops]

from datetime import date, timedelta, time
from dateutil.rrule import rrule, DAILY

a = datetime.now()
b = date(2020, 2, 27)

double_drops_dict = {}

for index, dt in enumerate(rrule(DAILY, dtstart=a, until=b)):
    double_drops_dict[dt.strftime("%Y-%m-%d")] = {
        "double_drops": double_drops[index],
        "remaining_stam": 12*24 - 15*3 - 20*2 - 25*2 + 20*2,
        "remaining_wings": 24
    }

hours_left_today = datetime.combine(date.today() + timedelta(days=1), time.min) - datetime.now()
hours_left_today = hours_left_today.seconds // 3600
double_drops_dict[datetime.now().strftime("%Y-%m-%d")]["remaining_stam"] = current_stam + 10*hours_left_today
double_drops_dict[datetime.now().strftime("%Y-%m-%d")]["remaining_wings"] = current_wings + hours_left_today 
double_drops_dict

{'2020-02-20': {'double_drops': ['wandering shroom',
   'catoblepas anemos',
   'eolian phantom',
   'proud manticore',
   'ebon chimera'],
  'remaining_stam': 86,
  'remaining_wings': 10},
 '2020-02-21': {'double_drops': ['wandering shroom'],
  'remaining_stam': 193,
  'remaining_wings': 24},
 '2020-02-22': {'double_drops': ['catoblepas anemos', 'proud manticore'],
  'remaining_stam': 193,
  'remaining_wings': 24},
 '2020-02-23': {'double_drops': ['wandering shroom'],
  'remaining_stam': 193,
  'remaining_wings': 24},
 '2020-02-24': {'double_drops': ['ebon chimera'],
  'remaining_stam': 193,
  'remaining_wings': 24},
 '2020-02-25': {'double_drops': ['catoblepas anemos', 'proud manticore'],
  'remaining_stam': 193,
  'remaining_wings': 24},
 '2020-02-26': {'double_drops': ['eolian phantom'],
  'remaining_stam': 193,
  'remaining_wings': 24}}

In [107]:
import copy
track_achieves = copy.deepcopy(achievements)
remaining_stamina = 0
remaining_wings = 0
for i in double_drops_dict.keys():
    print()
    print("Prospective Schedule for", i)
    print()
    todays_stam = double_drops_dict[i]["remaining_stam"]
    todays_wings = double_drops_dict[i]["remaining_wings"]
    print("Stamina:", todays_stam, "+", remaining_stamina)
    print("Wings:", todays_wings, "+", remaining_wings)
    todays_stam += remaining_stamina
    todays_wings += remaining_wings
    todays_double_drops = double_drops_dict[i]["double_drops"]
    prospective_fights = [i for i in track_achieves if i["remaining"] > 0 and i["title"] in todays_double_drops]
    print("Prospective Fights:\n\t", prospective_fights)
    stam_costs = [i["stam_cost"] for i in prospective_fights]
    stam_costs.sort(reverse=True)
    while stam_costs and todays_stam > min(stam_costs):
        for i in stam_costs:
            wing_cost = 3 if i == 30 else 2
            if i > todays_stam:
                if wing_cost > todays_wings:
                    continue
            for fight in track_achieves:
                if fight["stam_cost"] != i or fight["remaining"] <= 0 or not fight["title"] in todays_double_drops:
                    if fight["stam_cost"] == i and fight['remaining'] > 0 and not fight["title"] in todays_double_drops:
                        print(f"\t\tSkipping {fight['title']}")
                    continue
                runs = min(todays_stam // i, fight["remaining"])
                print(f"Run {runs} runs of {fight['title']} = {i*runs} stam")
                fight["remaining"] -= runs
                coops = min(todays_wings // wing_cost, fight["remaining"])
                print(f"Run {coops} coops of {fight['title']} = {wing_cost*coops} wings")
                fight["remaining"] -= coops
                break
            todays_stam = todays_stam - runs*i
            todays_wings = todays_wings - coops*wing_cost
    remaining_stamina = todays_stam if stam_costs else 0
    remaining_wings = min(todays_wings, 2)
    print(f"Spend {todays_wings - remaining_wings} wings on Dragon Trials!")
    if not stam_costs:
        print(f"Spend {todays_stam} on whatever extra stuff you want! But maybe don't spend honey, I included using the daily free honey.")
    print(f"Remaining Stamina: {remaining_stamina}; Remaining Wings: {remaining_wings}")
    print()

print([i for i in track_achieves if i['remaining'] > 0])
print()
print("Remaining fights:", [i["title"] for i in track_achieves if i["remaining"] > 0])
missing_stam = sum([i["stam_cost"] * i["remaining"] for i in track_achieves])
print("Required Stamina:", missing_stam, "=", missing_stam / 30, "big honeys.")


Prospective Schedule for 2020-02-20

Stamina: 86 + 0
Wings: 10 + 0
Prospective Fights:
	 [{'title': 'wandering shroom', 'stam_cost': 15, 'remaining': 20}, {'title': 'catoblepas anemos', 'stam_cost': 20, 'remaining': 20}, {'title': 'eolian phantom', 'stam_cost': 25, 'remaining': 18}, {'title': 'proud manticore', 'stam_cost': 25, 'remaining': 20}, {'title': 'ebon chimera', 'stam_cost': 30, 'remaining': 8}]
Run 2 runs of ebon chimera = 60 stam
Run 3 coops of ebon chimera = 9 wings
Run 1 runs of eolian phantom = 25 stam
Run 0 coops of eolian phantom = 0 wings
Spend 0 wings on Dragon Trials!
Remaining Stamina: 1; Remaining Wings: 1


Prospective Schedule for 2020-02-21

Stamina: 193 + 1
Wings: 24 + 1
Prospective Fights:
	 [{'title': 'wandering shroom', 'stam_cost': 15, 'remaining': 20}]
Run 12 runs of wandering shroom = 180 stam
Run 8 coops of wandering shroom = 16 wings
Spend 7 wings on Dragon Trials!
Remaining Stamina: 14; Remaining Wings: 2


Prospective Schedule for 2020-02-22

Stamina